In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Problem 1

#### 1a - Using ConcreteModel

In [2]:
model = ConcreteModel()

model.CROP = Set(initialize = ['cotton', 'corn'])

model.Num = Var(model.CROP, domain = NonNegativeIntegers)

model.fertilizer = Param(model.CROP, initialize = {'cotton': 6, 'corn': 2})
model.labor = Param(model.CROP, initialize = {'cotton': 5,'corn': 3})
model.profit = Param(model.CROP, initialize = {'cotton': 400, 'corn': 200})
model.total_workers = 150
model.total_fertilizers = 200
model.total_land = 50

def fertilizer_constraint(m, i):
    return quicksum(m.fertilizer[i] * m.Num[i] for i in m.CROP) <= m.total_fertilizers

def workers_constraint(m, i):
    return quicksum(m.labor[i] * m.Num[i] for i in m.CROP) <= m.total_workers

def total_land_constraint(m, i):
    return quicksum(m.Num[i] for i in model.CROP) <= m.total_land

def total_profit(m):
#     return sum_product(model.profit, model.Num, index = model.CROP)
    return quicksum(m.profit[i] * m.Num[i] for i in m.CROP)

model.fertilizer_constraint = Constraint(rule = fertilizer_constraint)
model.worker_constraint = Constraint(rule = workers_constraint)
model.land_constraint = Constraint(rule = total_land_constraint)

model.total_profit = Objective(rule = total_profit, sense = maximize)

In [3]:
opt = SolverFactory('glpk')
opt.solve(model)
for i in model.CROP:
    print(f"Number of acres of {i} to plant : ", value(model.Num[i]))

Number of acres of cotton to plant :  30.0
Number of acres of corn to plant :  0.0


#### 1b - Using AbstractModel

In [4]:
m = AbstractModel()

m.CROP = Set()

m.Num = Var(m.CROP, domain = NonNegativeIntegers)

m.fertilizer = Param(m.CROP)
m.labor = Param(m.CROP)
m.profit = Param(m.CROP)
m.total_workers = 150
m.total_fertilizers = 200
m.total_land = 50

def fertilizer_constraint(model, i):
    return quicksum(model.fertilizer[i] * model.Num[i] for i in model.CROP) <= m.total_fertilizers

def workers_constraint(model, i):
    return quicksum(model.labor[i] * model.Num[i] for i in model.CROP) <= m.total_workers

def total_land_constraint(model, i):
    return quicksum(model.Num[i] for i in model.CROP) <= model.total_land

def total_profit(model):
#     return sum_product(model.profit, model.Num, index = model.CROP)
    return quicksum(model.profit[i] * model.Num[i] for i in model.CROP)

m.fertilizer_constraint = Constraint(rule = fertilizer_constraint)
m.worker_constraint = Constraint(rule = workers_constraint)
m.land_constraint = Constraint(rule = total_land_constraint)

m.total_profit = Objective(rule = total_profit, sense = maximize)

In [5]:
df = pd.read_csv("hw2_p1b_data-1.csv")
instanceData = {None: {
    'CROP': {None: df["Plant"].unique()},
    'fertilizer' : df.set_index("Plant").to_dict()["fertilizerPerAcre"],
    'labor': df.set_index("Plant").to_dict()["workersPerAcre"],
    'profit': df.set_index("Plant").to_dict()["profitPerAcre"],
}}
instance = m.create_instance(instanceData)

In [6]:
opt = SolverFactory('glpk')
opt.solve(instance)
print("Total profit in light of the newly available options: ", instance.total_profit())

print("Difference in profit with newly available options: ", instance.total_profit() - model.total_profit())

Total profit in light of the newly available options:  19950.0
Difference in profit with newly available options:  7950.0


## Problem 2

#### Problem 2a - Using concrete model

In [7]:
pharma_model = ConcreteModel()

pharma_model.PLANT = Set(initialize = ['Ohio Valley', 'Lakeview'])
pharma_model.STORES = Set(initialize = ['Grand Rapids', 'Blue Ridge', 'Sunset'])

pharma_model.boxes = Var(pharma_model.PLANT, pharma_model.STORES, domain = NonNegativeReals)

pharma_model.cost = Param(pharma_model.PLANT, pharma_model.STORES, initialize = {
    ('Ohio Valley', 'Grand Rapids'): 50,
    ('Ohio Valley', 'Blue Ridge'): 40,
    ('Ohio Valley', 'Sunset'): 100,
    ('Lakeview', 'Grand Rapids'): 75,
    ('Lakeview', 'Blue Ridge'): 50,
    ('Lakeview', 'Sunset'): 75
})

pharma_model.production = Param(pharma_model.PLANT, initialize = {'Ohio Valley': 5000,'Lakeview': 7000})
pharma_model.demand = Param(pharma_model.STORES, initialize = {'Grand Rapids': 3000,'Blue Ridge': 5000, 'Sunset': 4000})


def production_constraint(m, p):
    return quicksum(m.boxes[p, s] for s in m.STORES) <= m.production[p]

def demand_constraint(m, s):
    return quicksum(m.boxes[p, s] for p in m.PLANT) >= m.demand[s]

def transportation_cost(m):
    return quicksum(m.cost[p, s] * m.boxes[p, s] for p in m.PLANT for s in m.STORES)

pharma_model.production_constraint = Constraint(pharma_model.PLANT, rule = production_constraint)
pharma_model.demand_constraint = Constraint(pharma_model.STORES, rule = demand_constraint)

pharma_model.transportation_cost = Objective(rule = transportation_cost, sense = minimize)

In [8]:
opt.solve(pharma_model)

from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for p in pharma_model.PLANT:
    for s in pharma_model.STORES:
        d[s][p] = value(pharma_model.boxes[p, s])  

In [9]:
print("Optimal Shipping plan: ")
pd.DataFrame(d).fillna('N/A')

Optimal Shipping plan: 


,Grand Rapids,Blue Ridge,Sunset
Ohio Valley,3000.0,2000.0,0.0
Lakeview,0.0,3000.0,4000.0


In [10]:
print("Total transport cost: ", pharma_model.transportation_cost())

Total transport cost:  680000.0


#### Problem 2b - Using concrete model

In [11]:
pharma_m = AbstractModel()

pharma_m.PLANT = Set()
pharma_m.STORES = Set()

pharma_m.boxes = Var(pharma_m.PLANT, pharma_m.STORES, domain = NonNegativeReals)

pharma_m.cost = Param(pharma_m.PLANT, pharma_m.STORES)

pharma_m.production = Param(pharma_m.PLANT)
pharma_m.demand = Param(pharma_m.STORES)

def production_constraint(m, p):
    return quicksum(m.boxes[p, s] for s in m.STORES) <= m.production[p]

def demand_constraint(m, s):
    return quicksum(m.boxes[p, s] for p in m.PLANT) == m.demand[s]

def transportation_cost(m):
    return quicksum(m.cost[p, s] * m.boxes[p, s] for p in m.PLANT for s in m.STORES)

pharma_m.production_constraint = Constraint(pharma_m.PLANT, rule = production_constraint)
pharma_m.demand_constraint = Constraint(pharma_m.STORES, rule = demand_constraint)

pharma_m.transportation_cost = Objective(rule = transportation_cost, sense = minimize)

In [12]:
df_1 = pd.read_csv("hw2_p2b_cost_data-1.csv")
df_2 = pd.read_csv("hw2_p2b_demand_data-1.csv")
df_3 = pd.read_csv("hw2_p2b_supply_data-1.csv")

In [13]:
instanceData = {None: {
    'PLANT': {None: df_3['Plant_ID'].unique()},
    'STORES': {None: df_2['Store_ID'].unique()},
    'demand': df_2.set_index('Store_ID').to_dict()['demand'],
    'production': df_3.set_index('Plant_ID').to_dict()['supply'],
    'cost': df_1.set_index(['Plant_ID', 'Store_ID']).to_dict()['cost']
}}

In [14]:
pharma_instance = pharma_m.create_instance(instanceData)

In [17]:
print("Total transport cost: ", pharma_instance.transportation_cost())

Total transport cost:  270160.0
